In [ ]:
!pip install neo4j

In [ ]:
from neo4j import GraphDatabase
import os
from google.colab import userdata

In [ ]:
# Connect to Neo4j (start Neo4j Desktop or AuraDB first)
# uri = "bolt://localhost:7687"
# driver = GraphDatabase.driver(uri, auth=("neo4j", "password"))

# Get Neo4j connection details from environment variables
uri = userdata.get('NEO4J_URI')
user = userdata.get('NEO4J_USER')
password = userdata.get('NEO4J_PASSWORD')

driver = GraphDatabase.driver(uri, auth=(user, password))


In [ ]:
# 1️⃣ Create a sample knowledge graph
def create_graph(tx):
    tx.run("""
        CREATE (m:Company {name:'Microsoft', founded:1975})
        CREATE (o:Company {name:'OpenAI', founded:2015})
        CREATE (p1:Person {name:'Bill Gates'})
        CREATE (p2:Person {name:'Sam Altman'})
        CREATE (m)-[:FOUNDED_BY]->(p1)
        CREATE (o)-[:FOUNDED_BY]->(p2)
    """)

with driver.session() as session:
    session.execute_write(create_graph)

In [ ]:
# 2️⃣ Query knowledge graph
def get_company_facts(tx, company_name):
    q = """
        MATCH (c:Company {name:$company})-[r:FOUNDED_BY]->(p:Person)
        RETURN c.name AS company, c.founded AS founded, collect(p.name) AS founders
    """
    result = tx.run(q, company=company_name)
    return [r.data() for r in result]

with driver.session() as session:
    facts = session.execute_read(get_company_facts, "OpenAI")

In [ ]:
# 3️⃣ Simple generation
def generate_answer(facts):
    if not facts:
        return "No data found."
    f = facts[0]
    return f"{f['company']} was founded in {f['founded']} by {', '.join(f['founders'])}."

print(generate_answer(facts))
driver.close()

SecretNotFoundError: Secret NEO4J_URI does not exist.